In [2]:
@which sum([1 2 3])

sum(a::AbstractArray; dims, kw...) in Base at reducedim.jl:873

In [5]:
using BenchmarkTools

In [6]:
function suma(A)
    s=0.0
    for a in A
        @fastmath s += a
    end
    s
end
    

suma (generic function with 1 method)

In [7]:
function sumas(A)
    s=0.0
    for a in A
        s+=a
    end
    s
end

sumas (generic function with 1 method)

In [8]:
a=rand(10^5)

100000-element Vector{Float64}:
 0.6486299512327511
 0.762123578251354
 0.995941615415582
 0.21900682904831825
 0.6416817842778444
 0.8136495859972988
 0.4133540331917005
 0.5782049465088326
 0.11953149434924248
 0.7977368742934459
 0.23144902325043715
 0.00048423340973036133
 0.987297965503301
 ⋮
 0.13092946567260033
 0.42226876823158
 0.027527659063180554
 0.9381657815877402
 0.2922622705525264
 0.529308816886479
 0.688019367956507
 0.3445136698625373
 0.10613260616311582
 0.7163849437588146
 0.7389006183892439
 0.894255151486796

In [13]:
bench_builtin = @benchmark sum(a)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  37.708 μs … 247.736 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     44.514 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   51.201 μs ±  16.840 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▃██▆▅▅▃▄▆▄▂   ▁▁▂▁           ▁                             ▂
  ▆▇████████████▇█████▇▇▆▅▇▇▆▇███████▇▅▆▆▆▇███▇▇▆▆▇▆▇▆▆▆▆▆▅▅▆▅ █
  37.7 μs       Histogram: log(frequency) by time       121 μs <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [14]:
bench_parallel = @benchmark suma(a)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  38.641 μs … 195.217 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     49.248 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.263 μs ±  12.489 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▂▃▆▆▆▆▇███▅▃▂▁▁▁▂▂▃▃▂▂▁               ▁ ▁                  ▂
  ▅██████████████████████████▇▇▇▇▆▆▇▆▇▅▇▇█████▇▆▇▅▅▇▇████▇█▆▅▆ █
  38.6 μs       Histogram: log(frequency) by time       102 μs <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [33]:
bench_serial= @benchmark sumas(a)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  165.568 μs … 445.901 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     227.292 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   229.860 μs ±  23.770 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▃      ▄▆▂          ▅█▆▄▄▅▅▄▃▃▄▃▃▃▃▂▂▂▂▁▁▁▁▁             ▂
  ▇▇▁▄▇██▅▁▃▁▄███▇▅▅▅▅▆▅▄▄▆█████████████████████████▇███▇▇▇▇▇▇▆ █
  166 μs        Histogram: log(frequency) by time        305 μs <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [30]:
d=Dict("Julia escrito a mano rápido" => minimum(bench_parallel.times)/1e6)

Dict{String, Float64} with 1 entry:
  "Julia escrito a mano rápido" => 0.038641

In [31]:
d["Julia built in"] = minimum(bench_builtin.times)/1e6

0.037708

In [37]:
d["Julia escrito a mano serial"]=minimum(bench_serial.times)/1e6

0.165568

In [38]:
d

Dict{String, Float64} with 3 entries:
  "Julia escrito a mano serial" => 0.165568
  "Julia escrito a mano rápido" => 0.038641
  "Julia built in"              => 0.037708

In [39]:
import Pkg

In [40]:
Pkg.add("DistributedArrays")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed Libtiff_jll ─────── v4.3.0+1
   Installed LERC_jll ────────── v3.0.0+1
   Installed Primes ──────────── v0.5.1
   Installed DistributedArrays ─ v0.6.6
    Updating `~/.julia/environments/v1.6/Project.toml`
  [aaf54ef3] + DistributedArrays v0.6.6
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [aaf54ef3] + DistributedArrays v0.6.6
  [27ebfcd6] + Primes v0.5.1
  [88015f11] + LERC_jll v3.0.0+1
  [89763e89] ↑ Libtiff_jll v4.3.0+0 ⇒ v4.3.0+1
Precompiling project...
  ✓ Primes
  ✓ LERC_jll
  ✓ DistributedArrays
  ✓ Libtiff_jll
  ✓ GR_jll
  ✓ GR
  ✓ Plots
  7 dependencies successfully precompiled in 115 seconds (152 already precompiled)


In [66]:
using Distributed
using DistributedArrays
addprocs(4)
@everywhere using DistributedArrays

In [69]:
adist =distribute(a)
j_bench_dist = @benchmark sum($adist)

BenchmarkTools.Trial: 1684 samples with 1 evaluation.
 Range (min … max):  2.492 ms …  17.665 ms  ┊ GC (min … max): 0.00% … 84.75%
 Time  (median):     2.850 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.930 ms ± 672.300 μs  ┊ GC (mean ± σ):  0.30% ±  2.07%

           ▁▃▇█▇▇▃                                             
  ▂▂▂▂▂▄▅▅▇████████▆▅▄▃▄▃▃▃▃▃▂▂▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▁▁▂▂▁▁▂ ▃
  2.49 ms         Histogram: frequency by time        4.08 ms <

 Memory estimate: 38.11 KiB, allocs estimate: 1019.

In [51]:
d["Julia 4x built-in"] = minimum(j_bench_dist.times)/1e6

2.590875

In [58]:
nprocs()

23

In [59]:
rmprocs(9:23,waitfor=0)

Task (runnable) @0x00007fc77447f9d0

In [72]:
nprocs()

16

In [64]:
bench_builtin = @benchmark sum(a)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  39.519 μs … 326.350 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     46.341 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.260 μs ±  16.231 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▇█▆▄▅▃▃▆▄▁     ▁                                          ▂
  ▇▇█████████████▇█████▇▇▆▇▇▇▇▇█▇███▇▇▆▇▆▆▇▇█▇▆▆▆▅▄▆▆▆▄▅▅▄▅▄▄▅ █
  39.5 μs       Histogram: log(frequency) by time       123 μs <

 Memory estimate: 16 bytes, allocs estimate: 1.

true

In [73]:
d

Dict{String, Float64} with 4 entries:
  "Julia escrito a mano serial" => 0.165568
  "Julia escrito a mano rápido" => 0.038641
  "Julia 4x built-in"           => 2.59088
  "Julia built in"              => 0.037708